# Big Query Machine Learning (BQML)

**Learning Objectives**
- Understand that it is possible to build ML models in Big Query
- Understand when this is appropriate
- Experience building a model using BQML

# Introduction

BigQuery is more than just a data warehouse, it also has some ML capabilities baked into it. 

As of January 2019 it is limited to linear models, but what it gives up in complexity, it gains in ease of use.

BQML is a great option when a linear model will suffice, or when you want a quick benchmark to beat, but for more complex models such as neural networks you will need to pull the data out of BigQuery and into an ML Framework like TensorFlow.

In this notebook, we will build a naive model using BQML. **This notebook is intended to inspire usage of BQML, we will not focus on model performance.**

### Set up environment variables and load necessary libraries

In [1]:
PROJECT = "qwiklabs-gcp-636667ae83e902b6"  # Replace with your PROJECT
REGION = "us-central1"            # Choose an available region for Cloud MLE

In [2]:
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [3]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Create BigQuery dataset

Prior to now we've just been reading an existing BigQuery table, now we're going to create our own so so we need some place to put it. In BigQuery parlance, `Dataset` means a folder for tables. 

We will take advantage of BigQuery's [Python Client](https://cloud.google.com/bigquery/docs/reference/libraries#client-libraries-install-python) to create the dataset.

In [4]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

dataset = bigquery.Dataset(bq.dataset("bqml_taxifare"))
try:
    bq.create_dataset(dataset) # will fail if dataset already exists
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset created


## Create model

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

#### **Exercise 1**
Use the query we created in the previous lab to Clean the Data to now train a Linear Regression model with BQML called `taxifare_model`. This should amount to adding a line to create the model and adding OPTIONS to specify the model type. Our label will be sum of `tolls_amount` and `fare_amount` and for features will use the pickup datetime and pickup & dropoff latitude and longitude.

HINT: Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) if you get stuck or if you want to see another example.

**Your query could take about two minutes to complete.**

In [7]:
%%bigquery --project $PROJECT
CREATE MODEL `bqml_taxifare.taxifare_model` 
OPTIONS (MODEL_TYPE='LINEAR_REG', input_label_cols=['fare_amount'])
AS SELECT
    (tolls_amount + fare_amount) AS fare_amount, -- create label that is the sum of fare_amount and tolls_amount
    pickup_datetime,
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude, 
    dropoff_latitude
FROM
    `nyc-tlc.yellow.trips`
WHERE
    -- Clean Data
    trip_distance > 0
    AND passenger_count > 0  
    AND fare_amount >= 2.5
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    
    -- create a repeatable 1/5000th sample
    AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 5000) = 1

""


## Get training statistics

Because the query uses a `CREATE MODEL` statement to create a table, you do not see query results. The output is an empty string.

To get the training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

#### **Exercise 2**

After completing the exercise above, query the training information of the model you created. Have a look at [Step Three and Four of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see a similar example.

In [9]:
%%bigquery --project $PROJECT
SELECT * FROM ML.TRAINING_INFO(MODEL `bqml_taxifare.taxifare_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,0,67.947115,61.568527,7737,None


In [12]:
import math

In [13]:
math.sqrt(61.56)

7.846018098373213

In [32]:
import pandas as pd

In [35]:
pd.set_option('max_colwidth', 500)

In [36]:
%%bigquery --project $PROJECT
SELECT * FROM 
ML.WEIGHTS(MODEL `bqml_taxifare.taxifare_model`)

,processed_input,weight,category_weights
0,pickup_datetime,-3.643556e-03,[]
1,_TS_DOM_pickup_datetime_,NaN,"[{'weight': 646618.6757129618, 'category': '24'}, {'weight': 646615.1595253181, 'category': '29'}, {'weight': 646633.1371889696, 'category': '2'}, {'weight': 646630.4916722543, 'category': '6'}, {'weight': 646626.0274167692, 'category': '13'}, {'weight': 646616.1175227228, 'category': '28'}, {'weight': 646613.8295574095, 'category': '31'}, {'weight': 646617.5070314493, 'category': '26'}, {'weight': 646624.1350161489, 'category': '16'}, {'weight': 646627.3523314188, 'category': '11'}, {'weigh..."
2,_TS_DOW_pickup_datetime_,NaN,"[{'weight': 369971.15618007845, 'category': '1'}, {'weight': 370917.84581466904, 'category': '4'}, {'weight': 370602.12490909104, 'category': '3'}, {'weight': 370286.6179063039, 'category': '2'}, {'weight': 371548.87724017736, 'category': '6'}, {'weight': 371233.3688703623, 'category': '5'}, {'weight': 371863.89146920486, 'category': '7'}]"
3,_TS_MOY_pickup_datetime_,NaN,"[{'weight': 4388.3264854227, 'category': '2'}, {'weight': 4409.370579535744, 'category': '1'}, {'weight': 4369.628783872192, 'category': '3'}, {'weight': 4308.765215676613, 'category': '6'}, {'weight': 4186.4676492563285, 'category': '12'}, {'weight': 4248.481018348726, 'category': '9'}, {'weight': 4349.027581483285, 'category': '4'}, {'weight': 4228.462078193821, 'category': '10'}, {'weight': 4329.139478892471, 'category': '5'}, {'weight': 4268.18113580154, 'category': '8'}, {'weight': 4288..."
4,_TS_HOD_pickup_datetime_,NaN,"[{'weight': 225731.1876937495, 'category': '23'}, {'weight': 225483.75771075927, 'category': '4'}, {'weight': 225600.5762688305, 'category': '13'}, {'weight': 225455.8613171645, 'category': '2'}, {'weight': 225534.53068032349, 'category': '8'}, {'weight': 225639.87100509342, 'category': '16'}, {'weight': 225429.54122350787, 'category': '0'}, {'weight': 225520.9194517654, 'category': '7'}, {'weight': 225704.72143538503, 'category': '21'}, {'weight': 225508.6838667417, 'category': '6'}, {'weig..."
5,_TS_MOH_pickup_datetime_,NaN,"[{'weight': -540907.1479721608, 'category': '4'}, {'weight': -540903.568241017, 'category': '21'}, {'weight': -540895.7312393712, 'category': '58'}, {'weight': -540895.9348238638, 'category': '57'}, {'weight': -540904.7338864933, 'category': '15'}, {'weight': -540896.1567458245, 'category': '56'}, {'weight': -540896.9910593614, 'category': '52'}, {'weight': -540902.219370283, 'category': '28'}, {'weight': -540902.4427232106, 'category': '26'}, {'weight': -540905.5489315494, 'category': '14'}..."
6,_TS_WOY_pickup_datetime_,NaN,"[{'weight': 741365.494898644, 'category': '32'}, {'weight': 714866.5485716852, 'category': '20'}, {'weight': 723699.1725898319, 'category': '24'}, {'weight': 763447.6302289588, 'category': '42'}, {'weight': 785531.8760352385, 'category': '52'}, {'weight': 747989.7353488865, 'category': '35'}, {'weight': 730323.8773089226, 'category': '27'}, {'weight': 706033.1359433094, 'category': '16'}, {'weight': 778907.229302743, 'category': '49'}, {'weight': 675116.3389469517, 'category': '2'}, {'weight..."
7,_TS_YEAR_pickup_datetime_,NaN,"[{'weight': -453106.1708813785, 'category': '2014'}, {'weight': -338518.128512094, 'category': '2015'}, {'weight': -567694.0760911808, 'category': '2013'}, {'weight': -1028255.1349192577, 'category': '2009'}, {'weight': -799079.2095685905, 'category': '2011'}, {'weight': -684491.3728818798, 'category': '2012'}, {'weight': -913667.1676550742, 'category': '2010'}]"
8,pickup_longitude,8.016266e+01,[]
9,pickup_latitude,-4.940728e+01,[]


'eval_loss' is reported as mean squared error. Your RMSE should be about **8.29**. Your results may vary.

## Predict

To use our model to make predictions, we use `ML.PREDICT`

#### **Exercise 3**

Lastly, use the `taxifare_model` you trained above to infer the cost of a taxi ride that occurs at 10:00 am on January 3rd, 2014 going
from the Google Office in New York (latitude: 40.7434, longitude: -74.0080) to the JFK airport (latitude: 40.6413, longitude: -73.7781)

**Hint**: Have a look at [Step Five of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) if you get stuck or if you want to see another example.

In [16]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2014-01-03 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,22.03639


In [20]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2015-01-03 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,21.917126


In [23]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2015-06-30 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,22.869389


In [27]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2015-10-01 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,23.413213


In [29]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2015-12-31 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,23.286171


In [30]:
%%bigquery --project $PROJECT
SELECT predicted_fare_amount 
FROM
ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
          (SELECT
          40.7434 as pickup_latitude,
          -74.0080 as pickup_longitude,
          40.6413 as dropoff_latitude,
          -73.7781 as dropoff_longitude,
           TIMESTAMP "2016-01-01 10:00:00" as pickup_datetime
          ))

,predicted_fare_amount
0,223952.713805


## Recap

The value of BQML is its ease of use:

- We created a model with just two additional lines of SQL
- We never had to move our data out of BigQuery
- We didn't need to use an ML Framework or code, just SQL

There's lots of work going on behind the scenes make this look easy. For example BQML is automatically creating a training/evaluation split, tuning our learning rate,  and one-hot encoding features if neccesary. When we move to TensorFlow these are all things we'll need to do ourselves. 

This notebook was just to inspire usagage of BQML, the current model is actually very poor. We'll prove this in the next lesson by beating it with a simple heuristic. 

We could improve our model considerably with some feature engineering but we'll save that for a future lesson. Also there are additional BQML functions such as `ML.WEIGHTS` and `ML.EVALUATE` that we haven't even explored. If you're interested in learning more about BQML I encourage you to [read the offical docs](https://cloud.google.com/bigquery/docs/bigqueryml).

From here on out we'll focus on pulling data out of BigQuery and building models using TensorFlow, which is more effort but also offers much more flexibility.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.